# Sistema 4

## Micela
En este último problema trataremos de simular el proceso de formación de una micela, es decir, un sistema coloidal formado por moléculas tensioactivas en un disolvente. Las micelas en agua típicamente se forman a partir del agregado de moléculas anfipáticas, es decir, aquellas con un extremo hidrofílico (cabeza polar) y otro hidrofóbico (cola apolar) que aislan la parte apolar en el interior de la nueva estructura mientras exponen su cabeza polar en el agua, otorgándoles solubilidad en el disolvente. Un ejemplo es la sal de ácido oleico.

A partir de las funciones aprendidas en los anteriores problemas, prepararemos un sistema con <tt>N_Polimers</tt> de <tt>N_Beads</tt>. Definimos el primer monómero o bead como <tt>type_Q</tt>, de carga negativa, mientras el resto de monómeros de la cadena están definidos como <tt>type_A</tt>, sin carga. A partir de las interacciones no-enlazantes (Lennard-Jones y Debye-Hückel) los polímeros se agregarán para formar las micelas gradualmente. Fíjate en el avance de la simulación para ver su proceso de formación.

In [1]:
import espressomd
from espressomd import polymer
from espressomd.interactions import HarmonicBond
import espressomd.electrostatics
import numpy as np
import tipus_particules as tp

In [2]:
#Dimensiones del sistema:
system=espressomd.System(box_l=[6,6,6],periodicity=[False,False,False])   # Inicializamos el sistema. Caja de 6x6x6 nm3


# Parámetros del sistema
temperatura = 300
R           = 8.314472
RT          = temperatura * R

N_Polimers=20
N_Beads=8
dist_Beads=0.5   # nm
Semilla = 20

In [3]:
#Partículas del sistema

#Código numérico de los tipos de partículas.
type_A = 0     
type_Q = 2

#Parámetros de carga
charges = {}
charges[type_A] = 0
charges[type_Q] = -1.

#Parámetros de masa en kg/mol
mass = {}
mass[type_A] = 0.050   # En kg/mol
mass[type_Q] = 0.050   # En kg/mol

# 
D_coeff = {}  # En nm2/ns
D_coeff[type_A] = 1.0      # 
D_coeff[type_Q] = 1.0      # 


# Fricciones. Isotrópicas 

Fric={}
Fric[type_A]=[RT/D_coeff[type_A]]*3
Fric[type_Q]=[RT/D_coeff[type_Q]]*3

In [4]:
# Generamos los polímeros

#Coordenadas aleatorias iniciales
polymers_xyz = polymer.positions(n_polymers=N_Polimers,
                             beads_per_chain=N_Beads,
                             bond_length=dist_Beads, seed=Semilla)

#Parámetros del enlace 
k_bond_CC=7.6E6        #     en J/(mol nm2) = 317 kcal/(molA2)
hb = HarmonicBond(k=k_bond_CC, r_0=dist_Beads)   # Unidades J/molnm2 i N: nm
system.bonded_inter.add(hb)


for polymer in polymers_xyz:
    for index, position in enumerate(polymer):
        id = len(system.part)
    
        if index==0:  # El primer índice: site Cargado
            system.part.add(id=id, pos=position, type=type_Q, q=charges[type_Q]
                        ,mass=mass[type_Q],gamma=Fric[type_Q])  
        else:         # resto de sites de tipo A
            system.part.add(id=id, pos=position, type=type_A, q=charges[type_A]
                        ,mass=mass[type_A],gamma=Fric[type_A])
        if index > 0:  # Añadir un enlace entre partículas id y id-1
            system.part[id].add_bond((hb, id - 1))    

In [5]:
# Interacciones
    
lj_epsilon=5000.
lj_sigma=0.5
lj_cutoff=2.4
lj_shift=0.
r_cut_DH=system.box_l[0]*0.4


system.non_bonded_inter[type_A, type_A].lennard_jones.set_params(
                epsilon=lj_epsilon, sigma=lj_sigma, cutoff=lj_cutoff, shift=lj_shift)    
    
system.non_bonded_inter[type_Q, type_A].lennard_jones.set_params(
                epsilon=lj_epsilon, sigma=lj_sigma, cutoff=lj_cutoff, shift=lj_shift)    

system.non_bonded_inter[type_Q, type_Q].lennard_jones.set_params(
                epsilon=lj_epsilon/100, sigma=lj_sigma, cutoff=lj_cutoff, shift=lj_shift)    
  
# Variables Medi
kappa_gas=138072.   # en J/(mol nm)   Prefactor de Coulomb en fase gas

# Constante dieléctrica del disolvente. Ej. Agua
die_solvent=80

prefactor_val=kappa_gas/die_solvent    
kappa_DH=0   # Si 0 Sin fuerza iónica. Para kappa_DH=0 corresponde a Coulombico puro    
        
 
DHsolv = espressomd.electrostatics.DH(prefactor=prefactor_val,kappa=kappa_DH,r_cut=r_cut_DH,check_neutrality=False)
system.actors.add(DHsolv)   

In [6]:
# Minimización energética
force_tolerance  = 0.1
max_steps        = 5000
max_displacement = 0.1 * lj_sigma
damping          = 30
Emin_per_step    = 1000

system.time_step =  0.000003    # ns
system.cell_system.skin =  0.3  

system.integrator.set_steepest_descent(f_max=0, gamma=damping, max_displacement=max_displacement)
system.integrator.run(0)
old_force = np.max(np.linalg.norm(system.part[:].f, axis=1))

while system.time / system.time_step < max_steps:
    system.integrator.run(Emin_per_step)
    force = np.max(np.linalg.norm(system.part[:].f, axis=1))
    rel_force = np.abs((force - old_force) / old_force)
    print(f'rel. force change:{rel_force:.2e}, force:{force:.2e}')
    if rel_force < force_tolerance :
        break
    old_force = force

rel. force change:1.00e+00, force:6.45e+07
rel. force change:0.00e+00, force:6.45e+07


In [7]:
# Activamos el Modelo Velocity-Verlet
system.integrator.set_vv()
# Activamos el termostato
system.thermostat.set_langevin(kT=RT, gamma=1 , seed=Semilla)

In [8]:
import datetime
inicio = datetime.datetime.now()

In [9]:
niter=500
pasos=2000

tp.save_vxyz(system,'Trajectory4_NPoly20_NBeads8.xyz','w',aplicar_PBC=False)


for it in range(pasos):
    system.integrator.run(niter)                  # Realiza niter de dinámica
    tp.save_vxyz(system,'Trajectory4_NPoly20_NBeads8.xyz','a',aplicar_PBC=False)    # Save coordenada
    if it%100==0:
        print('Simulando {} de {} pasos'.format(it,pasos))

Simulando 0 de 2000 pasos
Simulando 100 de 2000 pasos
Simulando 200 de 2000 pasos
Simulando 300 de 2000 pasos
Simulando 400 de 2000 pasos
Simulando 500 de 2000 pasos
Simulando 600 de 2000 pasos
Simulando 700 de 2000 pasos
Simulando 800 de 2000 pasos
Simulando 900 de 2000 pasos
Simulando 1000 de 2000 pasos
Simulando 1100 de 2000 pasos
Simulando 1200 de 2000 pasos
Simulando 1300 de 2000 pasos
Simulando 1400 de 2000 pasos
Simulando 1500 de 2000 pasos
Simulando 1600 de 2000 pasos
Simulando 1700 de 2000 pasos
Simulando 1800 de 2000 pasos
Simulando 1900 de 2000 pasos


In [10]:
fin = datetime.datetime.now()
print('Tiempo de cálculo {}'.format(fin-inicio))

Tiempo de cálculo 0:03:06.066180


# Práctica
- Estudia la formación de una micela a partir de un modelo sencillo de tensioactivo, cabeza polar y cola hidrofóbica.
- Analiza el efecto del tamaño del tensioactivo.